In [26]:
import csv
import random
from autogluon.tabular import TabularDataset, TabularPredictor
import os
property_label = "P19"
orientation = "objects"
limit = "full"

In [18]:
import random

# load instance dataset
# p50_instances = {}
# tsv_file = open(f'../output/allConstraintsAnalysis_Final/instances/instances.subjects.P50.tsv', encoding="utf-8")
# read_tsv = csv.reader(tsv_file, delimiter="\t")
# for line in read_tsv:
#     if line[0] not in p50_instances:
#         p50_instances[line[0]] = [line[1]]
#     else:
#         p50_instances[line[0]].append(line[1])

# p19_instances = {}
# tsv_file = open(f'../output/allConstraintsAnalysis_Final/instances/instances.objects.P19.tsv', encoding="utf-8")
# read_tsv = csv.reader(tsv_file, delimiter="\t")
# for line in read_tsv:
#     if line[0] not in p19_instances:
#         p19_instances[line[0]] = [line[1]]
#     else:
#         p19_instances[line[0]].append(line[1])

p39_instances = {}
tsv_file = open(f'../output/allConstraintsAnalysis_Final/instances/instances.objects.P39.tsv', encoding="utf-8")
read_tsv = csv.reader(tsv_file, delimiter="\t")
for line in read_tsv:
    if line[0] not in p39_instances:
        p39_instances[line[0]] = [line[1]]
    else:
        p39_instances[line[0]].append(line[1])

# p161_instances = {}
# tsv_file = open(f'../output/allConstraintsAnalysis_Final/instances/instances.subjects.P161.tsv', encoding="utf-8")
# read_tsv = csv.reader(tsv_file, delimiter="\t")
# for line in read_tsv:
#     if line[0] not in p161_instances:
#         p161_instances[line[0]] = [line[1]]
#     else:
#         p161_instances[line[0]].append(line[1])

# p50_objects = set()
# tsv_file = open(f'../data/propertiesSplit_final/claims.P50.tsv')
# read_tsv = csv.reader(tsv_file, delimiter="\t")
# next(read_tsv)
# for line in read_tsv:
#     if line[0] in p50_instances:
#         p50_objects.add(line[0])
# p50_objects = random.sample(list(p50_objects),250000)
# p50_class_map = {}
# for cla in p50_objects:
#     p50_class_map[cla] = 1

# p161_objects = set()
# tsv_file = open(f'../data/propertiesSplit_final/claims.P161.tsv')
# read_tsv = csv.reader(tsv_file, delimiter="\t")
# next(read_tsv)
# for line in read_tsv:
#     if line[0] in p161_instances:
#         p161_objects.add(line[0])
# p161_class_map = {}
# for cla in p161_objects:
#     p161_class_map[cla] = 1
    
p39_objects = set()
tsv_file = open(f'../data/propertiesSplit_final/claims.P39.tsv')
read_tsv = csv.reader(tsv_file, delimiter="\t")
next(read_tsv)
for line in read_tsv:
    if line[2] in p39_instances:
        p39_objects.add(line[2])
p39_class_map = {}
for cla in p39_objects:
    p39_class_map[cla] = 1
    
# p19_objects = set()
# tsv_file = open(f'../data/propertiesSplit_final/claims.P19.tsv')
# read_tsv = csv.reader(tsv_file, delimiter="\t")
# next(read_tsv)
# for line in read_tsv:
#     if line[2] in p19_instances:
#         p19_objects.add(line[2])
# p19_class_map = {}
# for cla in p19_objects:
#     p19_class_map[cla] = 1
    
# new_p50_map = {}
# for key in p50_instances:
#     if key in p50_class_map:
#         new_p50_map[key] = p50_instances[key]
# p50_instances = new_p50_map

#print(len(p19_class_map))
print(len(p39_class_map))
# print(len(p50_class_map))
# print(len(p161_class_map))
# print(len(p19_instances))
print(len(p39_instances))
#print(len(p50_instances))
#print(len(p161_instances))

103855
103856


In [19]:
# build transE embeddings dataset

p19_list = []
p50_list = []
p161_list = []
p39_list = []

with open("../embeddings/profile_graph_embeddings.TransE.tsv") as file:
    tsv_file = csv.reader(file, delimiter=" ")
    next(tsv_file)
    for line in tsv_file:
#         if line[0] in p19_instances:
#             instances_to_use = p19_instances[line[0]]
#             newline = line[:]
#             newline.append(random.choice(instances_to_use))
#             assert len(newline) == 102, newline
#             p19_list.append(newline)
#         if line[0] in p50_instances:
#             instances_to_use = p50_instances[line[0]]
#             newline = line[:]
#             newline.append(random.choice(instances_to_use))
#             assert len(newline) == 102, newline
#             p50_list.append(newline)
#         if line[0] in p161_instances:
#             instances_to_use = p161_instances[line[0]]
#             newline = line[:]
#             newline.append(random.choice(instances_to_use))
#             assert len(newline) == 102, newline
#             p161_list.append(newline)
        if line[0] in p39_instances:
            instances_to_use = p39_instances[line[0]]
            newline = line[:]
            newline.append(random.choice(instances_to_use))
            assert len(newline) == 102, newline
            p39_list.append(newline)

# with open(f"../output/instance_prediction_datasets/250000/P19.objects.transE.embeddings.{limit}.tsv", 'wt',encoding="utf-8",newline='') as out_file:
#     tsv_writer = csv.writer(out_file, delimiter='\t')
#     tsv_writer.writerow(["Qnode","pos1","pos2","pos3","pos4","pos5","pos6","pos7","pos8","pos9","pos10","pos11","pos12","pos13","pos14","pos15","pos16","pos17","pos18","pos19","pos20","pos21","pos22","pos23","pos24","pos25","pos26","pos27","pos28","pos29","pos30","pos31","pos32","pos33","pos34","pos35","pos36","pos37","pos38","pos39","pos40","pos41","pos42","pos43","pos44","pos45","pos46","pos47","pos48","pos49","pos50","pos51","pos52","pos53","pos54","pos55","pos56","pos57","pos58","pos59","pos60","pos61","pos62","pos63","pos64","pos65","pos66","pos67","pos68","pos69","pos70","pos71","pos72","pos73","pos74","pos75","pos76","pos77","pos78","pos79","pos80","pos81","pos82","pos83","pos84","pos85","pos86","pos87","pos88","pos89","pos90","pos91","pos92","pos93","pos94","pos95","pos96","pos97","pos98","pos99","pos100","Yval"])
#     tsv_writer.writerows(p19_list)
with open(f"../output/instance_prediction_datasets/250000/P39.objects.transE.embeddings.{limit}.tsv", 'wt',encoding="utf-8",newline='') as out_file:
    tsv_writer = csv.writer(out_file, delimiter='\t')
    tsv_writer.writerow(["Qnode","pos1","pos2","pos3","pos4","pos5","pos6","pos7","pos8","pos9","pos10","pos11","pos12","pos13","pos14","pos15","pos16","pos17","pos18","pos19","pos20","pos21","pos22","pos23","pos24","pos25","pos26","pos27","pos28","pos29","pos30","pos31","pos32","pos33","pos34","pos35","pos36","pos37","pos38","pos39","pos40","pos41","pos42","pos43","pos44","pos45","pos46","pos47","pos48","pos49","pos50","pos51","pos52","pos53","pos54","pos55","pos56","pos57","pos58","pos59","pos60","pos61","pos62","pos63","pos64","pos65","pos66","pos67","pos68","pos69","pos70","pos71","pos72","pos73","pos74","pos75","pos76","pos77","pos78","pos79","pos80","pos81","pos82","pos83","pos84","pos85","pos86","pos87","pos88","pos89","pos90","pos91","pos92","pos93","pos94","pos95","pos96","pos97","pos98","pos99","pos100","Yval"])
    tsv_writer.writerows(p39_list)
# with open(f"../output/instance_prediction_datasets/250000/P161.subjects.transE.embeddings.{limit}.tsv", 'wt',encoding="utf-8",newline='') as out_file:
#     tsv_writer = csv.writer(out_file, delimiter='\t')
#     tsv_writer.writerow(["Qnode","pos1","pos2","pos3","pos4","pos5","pos6","pos7","pos8","pos9","pos10","pos11","pos12","pos13","pos14","pos15","pos16","pos17","pos18","pos19","pos20","pos21","pos22","pos23","pos24","pos25","pos26","pos27","pos28","pos29","pos30","pos31","pos32","pos33","pos34","pos35","pos36","pos37","pos38","pos39","pos40","pos41","pos42","pos43","pos44","pos45","pos46","pos47","pos48","pos49","pos50","pos51","pos52","pos53","pos54","pos55","pos56","pos57","pos58","pos59","pos60","pos61","pos62","pos63","pos64","pos65","pos66","pos67","pos68","pos69","pos70","pos71","pos72","pos73","pos74","pos75","pos76","pos77","pos78","pos79","pos80","pos81","pos82","pos83","pos84","pos85","pos86","pos87","pos88","pos89","pos90","pos91","pos92","pos93","pos94","pos95","pos96","pos97","pos98","pos99","pos100","Yval"])
#     tsv_writer.writerows(p161_list)
# with open(f"../output/instance_prediction_datasets/250000/P50.subjects.transE.embeddings.{limit}.tsv", 'wt',encoding="utf-8",newline='') as out_file:
#     tsv_writer = csv.writer(out_file, delimiter='\t')
#     tsv_writer.writerow(["Qnode","pos1","pos2","pos3","pos4","pos5","pos6","pos7","pos8","pos9","pos10","pos11","pos12","pos13","pos14","pos15","pos16","pos17","pos18","pos19","pos20","pos21","pos22","pos23","pos24","pos25","pos26","pos27","pos28","pos29","pos30","pos31","pos32","pos33","pos34","pos35","pos36","pos37","pos38","pos39","pos40","pos41","pos42","pos43","pos44","pos45","pos46","pos47","pos48","pos49","pos50","pos51","pos52","pos53","pos54","pos55","pos56","pos57","pos58","pos59","pos60","pos61","pos62","pos63","pos64","pos65","pos66","pos67","pos68","pos69","pos70","pos71","pos72","pos73","pos74","pos75","pos76","pos77","pos78","pos79","pos80","pos81","pos82","pos83","pos84","pos85","pos86","pos87","pos88","pos89","pos90","pos91","pos92","pos93","pos94","pos95","pos96","pos97","pos98","pos99","pos100","Yval"])
#     tsv_writer.writerows(p50_list)

In [5]:
# build text embeddings dataset
import zipfile, io, gzip

# dimensionality = 1024
with open(f"../output/instance_prediction_datasets/250000/all.objects.text.embeddings.{limit}.tsv", 'wt',encoding="utf-8",newline='') as out_file:
    tsv_writer = csv.writer(out_file, delimiter='\t')
    for file in os.listdir("../embeddings/text_embeddings"):
        print(file)
        with zipfile.ZipFile("../embeddings/text_embeddings/"+file) as zfile:
            for name in zfile.namelist():
                print(name)
                zfiledata = io.BytesIO(zfile.read(name))
                with gzip.open(zfiledata, 'rb') as f:
                    lines = f.readlines()
                    for line in lines:
                        bytestring = line.decode("utf-8").split("\t")
                        assert " " not in bytestring[0], len(bytestring[0])
                        if bytestring[0].startswith("Q"):
                            qnode = bytestring[0]
#                             if qnode in p19_instances:
#                                 instances_to_use = p19_instances[qnode]
#                                 newline = [qnode]
#                                 for embedding in bytestring[2].split(","):
#                                     newline.append(embedding.replace("\r","").replace("\n","").replace("\"",""))
#                                 newline.append(random.choice(instances_to_use))
#                                 newline.append("P19")
#                                 assert len(newline) == 1027, newline
#                                 tsv_writer.writerow(newline)
                            if qnode in p50_instances:
                                instances_to_use = p50_instances[qnode]
                                newline = [qnode]
                                for embedding in bytestring[2].split(","):
                                    newline.append(embedding.replace("\r","").replace("\n","").replace("\"",""))
                                newline.append(random.choice(instances_to_use))
                                newline.append("P50")
                                assert len(newline) == 1027, newline
                                tsv_writer.writerow(newline)
#                             if qnode in p161_instances:
#                                 instances_to_use = p161_instances[qnode]
#                                 newline = [qnode]
#                                 for embedding in bytestring[2].split(","):
#                                     newline.append(embedding.replace("\r","").replace("\n","").replace("\"",""))
#                                 newline.append(random.choice(instances_to_use))
#                                 newline.append("P161")
#                                 assert len(newline) == 1027, newline
#                                 tsv_writer.writerow(newline)
#                             if qnode in p39_instances:
#                                 instances_to_use = p39_instances[qnode]
#                                 newline = [qnode]
#                                 for embedding in bytestring[2].split(","):
#                                     newline.append(embedding.replace("\r","").replace("\n","").replace("\"",""))
#                                 newline.append(random.choice(instances_to_use))
#                                 newline.append("P39")
#                                 assert len(newline) == 1027, newline
#                                 tsv_writer.writerow(newline)

text-embeddings-20220406T185201Z-001.zip
text-embeddings/kgtk_text_embeddings87.tsv.gz
text-embeddings/kgtk_text_embeddings98.tsv.gz
text-embeddings/kgtk_text_embeddings264.tsv.tsv.gz
text-embeddings-20220406T185201Z-002.zip
text-embeddings/kgtk_text_embeddings81.tsv.gz
text-embeddings/kgtk_text_embeddings83.tsv.gz
text-embeddings-20220406T185201Z-003.zip
text-embeddings/kgtk_text_embeddings92.tsv.gz
text-embeddings/kgtk_text_embeddings82.tsv.gz
text-embeddings-20220406T185201Z-004.zip
text-embeddings/kgtk_text_embeddings79.tsv.gz
text-embeddings/kgtk_text_embeddings84.tsv.gz
text-embeddings-20220406T185201Z-005.zip
text-embeddings/kgtk_text_embeddings8.tsv.gz
text-embeddings/kgtk_text_embeddings97.tsv.gz
text-embeddings-20220406T185201Z-006.zip
text-embeddings/kgtk_text_embeddings93.tsv.gz
text-embeddings-20220406T185201Z-007.zip
text-embeddings/kgtk_text_embeddings7.tsv.gz
text-embeddings/kgtk_text_embeddings75.tsv.gz
text-embeddings-20220406T185201Z-008.zip
text-embeddings/kgtk_text

text-embeddings-20220406T185201Z-060.zip
text-embeddings/kgtk_text_embeddings210.tsv.gz
text-embeddings/kgtk_text_embeddings73.tsv.gz
text-embeddings-20220406T185201Z-061.zip
text-embeddings/kgtk_text_embeddings219.tsv.gz
text-embeddings/kgtk_text_embeddings212.tsv.gz
text-embeddings/kgtk_text_embeddings264.tsv.gz
text-embeddings-20220406T185201Z-062.zip
text-embeddings/kgtk_text_embeddings211.tsv.gz
text-embeddings/kgtk_text_embeddings58.tsv.gz
text-embeddings-20220406T185201Z-063.zip
text-embeddings/kgtk_text_embeddings207.tsv.gz
text-embeddings/kgtk_text_embeddings206.tsv.gz
text-embeddings-20220406T185201Z-064.zip
text-embeddings/kgtk_text_embeddings214.tsv.gz
text-embeddings/kgtk_text_embeddings204.tsv.gz
text-embeddings-20220406T185201Z-065.zip
text-embeddings/kgtk_text_embeddings20.tsv.gz
text-embeddings/kgtk_text_embeddings43.tsv.gz
text-embeddings/kgtk_text_embeddings230.tsv.gz
text-embeddings-20220406T185201Z-066.zip
text-embeddings/kgtk_text_embeddings209.tsv.gz
text-embeddi

text-embeddings-20220406T185201Z-124.zip
text-embeddings/kgtk_text_embeddings139.tsv.gz
text-embeddings/kgtk_text_embeddings165.tsv.gz
text-embeddings-20220406T185201Z-125.zip
text-embeddings/kgtk_text_embeddings13.tsv.gz
text-embeddings-20220406T185201Z-126.zip
text-embeddings/kgtk_text_embeddings11.tsv.gz
text-embeddings-20220406T185201Z-127.zip
text-embeddings/kgtk_text_embeddings123.tsv.gz
text-embeddings/kgtk_text_embeddings110.tsv.gz
text-embeddings-20220406T185201Z-128.zip
text-embeddings/kgtk_text_embeddings124.tsv.gz
text-embeddings/kgtk_text_embeddings126.tsv.gz
text-embeddings-20220406T185201Z-129.zip
text-embeddings/kgtk_text_embeddings111.tsv.gz
text-embeddings-20220406T185201Z-130.zip
text-embeddings/kgtk_text_embeddings105.tsv.gz
text-embeddings/kgtk_text_embeddings10.tsv.gz
text-embeddings-20220406T185201Z-131.zip
text-embeddings/kgtk_text_embeddings106.tsv.gz
text-embeddings-20220406T185201Z-132.zip
text-embeddings/kgtk_text_embeddings107.tsv.gz
text-embeddings/kgtk_te

In [12]:
# separate large data file into 4 separate files
with open(f"../output/instance_prediction_datasets/250000/P50.objects.text.embeddings.{limit}.tsv", 'wt',encoding="utf-8",newline='') as p50_file, open(f"../output/instance_prediction_datasets/250000/P19.objects.text.embeddings.{limit}.tsv", 'wt',encoding="utf-8",newline='') as p19_file,open(f"../output/instance_prediction_datasets/250000/P39.objects.text.embeddings.{limit}.tsv", 'wt',encoding="utf-8",newline='') as p39_file,open(f"../output/instance_prediction_datasets/250000/P161.objects.text.embeddings.{limit}.tsv", 'wt',encoding="utf-8",newline='') as p161_file: 
    p50_writer = csv.writer(p50_file, delimiter='\t')
    p19_writer = csv.writer(p19_file, delimiter='\t')
    p39_writer = csv.writer(p39_file, delimiter='\t')
    p161_writer = csv.writer(p161_file, delimiter='\t')
    with open(f"../output/instance_prediction_datasets/250000/all.objects.text.embeddings.{limit}.tsv",encoding="utf-8") as file:
        tsv_file = csv.reader(file, delimiter="\t")
        for line in tsv_file:
            if line[-1] == "P50":
                p50_writer.writerow(line[:-1])
            if line[-1] == "P39":
                p39_writer.writerow(line[:-1])
            if line[-1] == "P19":
                p19_writer.writerow(line[:-1])
            if line[-1] == "P161":
                p161_writer.writerow(line[:-1])

In [12]:
# limit large p50 file
with open(f"../output/instance_prediction_datasets/250000/P50.objects.text.embeddings.{limit}.new.tsv", 'wt',encoding="utf-8",newline='') as p50_output:
    tsv_writer = csv.writer(p50_output, delimiter="\t")
    with open(f"../output/instance_prediction_datasets/250000/P50.subjects.text.embeddings.{limit}.tsv",encoding="utf-8") as p50_file:
        tsv_reader = csv.reader(p50_file, delimiter="\t")
        for line in tsv_reader:
            if line[0] in p50_instances:
                tsv_writer.writerow(line)

In [ ]:
# get top 10 classes from data
all_classes = set()
instance_count = {}
with open(f"../output/instance_prediction_datasets/{property_label}.{orientation}.1250.tsv",encoding="utf-8") as file:
    tsv_file = csv.reader(file, delimiter="\t")
    next(tsv_file)
    for line in tsv_file:
        all_classes.add(line[0])
for qnode in all_classes:
    instance_list = instances[qnode]
    for this_instance in instance_list:
        if this_instance in instance_count:
            instance_count[this_instance]+=1
        else:
            instance_count[this_instance]=1
print({k: v for k, v in sorted(instance_count.items(), key=lambda item: item[1], reverse=True)})

In [ ]:
# filter dataset for top 10 classes
classes_to_use = set()
lines_to_write = []
top_10_classes = ["Q11424","Q5398426","Q506240","Q7777570","Q21191270","Q20667187","Q229390","Q43099869","Q15416","Q61220733"]
with open(f"../output/instance_prediction_datasets/{property_label}.{orientation}.1250.tsv",encoding="utf-8") as file:
    tsv_file = csv.reader(file, delimiter="\t")
    next(tsv_file)
    for line in tsv_file:
        for this_instance in instances[line[0]]:
            if this_instance in top_10_classes:
                lines_to_write.append(line)
                classes_to_use.add(line[0])
print(len(classes_to_use))
# with open(f"../output/instance_prediction_datasets/{property_label}.{orientation}.top10classes.tsv", 'wt',encoding="utf-8") as out_file:
#     tsv_writer = csv.writer(out_file, delimiter='\t')
#     tsv_writer.writerow(["node","nodeLabel","prop","nodeConnection","connectionLabel","nodeOrientation"])
#     tsv_writer.writerows(lines_to_write)

In [21]:
lines = []
class_count = {}
lines_to_write = []
with open(f"../output/instance_prediction_datasets/250000/{property_label}.{orientation}.transE.embeddings.{limit}.tsv") as file:
    tsv_reader = csv.reader(file, delimiter="\t")
    next(tsv_reader)
    for line in tsv_reader:
        lines.append(line)
        if line[-1] in class_count:
            class_count[line[-1]] += 1
        else:
            class_count[line[-1]] = 1

for line in lines:
    if class_count[line[-1]] >= 10:
        lines_to_write.append(line)
            
with open(f"../output/instance_prediction_datasets/250000/{property_label}.{orientation}.transE.embeddings.{limit}.commonclasses.tsv", 'wt',encoding="utf-8",newline='') as out_file:
    tsv_writer = csv.writer(out_file, delimiter='\t')
    tsv_writer.writerow(["Qnode","pos1","pos2","pos3","pos4","pos5","pos6","pos7","pos8","pos9","pos10","pos11","pos12","pos13","pos14","pos15","pos16","pos17","pos18","pos19","pos20","pos21","pos22","pos23","pos24","pos25","pos26","pos27","pos28","pos29","pos30","pos31","pos32","pos33","pos34","pos35","pos36","pos37","pos38","pos39","pos40","pos41","pos42","pos43","pos44","pos45","pos46","pos47","pos48","pos49","pos50","pos51","pos52","pos53","pos54","pos55","pos56","pos57","pos58","pos59","pos60","pos61","pos62","pos63","pos64","pos65","pos66","pos67","pos68","pos69","pos70","pos71","pos72","pos73","pos74","pos75","pos76","pos77","pos78","pos79","pos80","pos81","pos82","pos83","pos84","pos85","pos86","pos87","pos88","pos89","pos90","pos91","pos92","pos93","pos94","pos95","pos96","pos97","pos98","pos99","pos100","Yval"])
    tsv_writer.writerows(lines_to_write)
    

In [15]:
with open(f"../output/instance_prediction_datasets/250000/P50.subjects.text.embeddings.full.tsv", encoding="utf-8") as fr, open(f"../output/instance_prediction_datasets/250000/P50.subjects.text.embeddings.header.{limit}.tsv",'wt',encoding="utf-8",newline='') as fw:
    cr = csv.reader(fr, delimiter='\t')
    cw = csv.writer(fw, delimiter='\t')
    cw.writerow(["class","emb1","emb2","emb3","emb4","emb5","emb6","emb7","emb8","emb9","emb10","emb11","emb12","emb13","emb14","emb15","emb16","emb17","emb18","emb19","emb20","emb21","emb22","emb23","emb24","emb25","emb26","emb27","emb28","emb29","emb30","emb31","emb32","emb33","emb34","emb35","emb36","emb37","emb38","emb39","emb40","emb41","emb42","emb43","emb44","emb45","emb46","emb47","emb48","emb49","emb50","emb51","emb52","emb53","emb54","emb55","emb56","emb57","emb58","emb59","emb60","emb61","emb62","emb63","emb64","emb65","emb66","emb67","emb68","emb69","emb70","emb71","emb72","emb73","emb74","emb75","emb76","emb77","emb78","emb79","emb80","emb81","emb82","emb83","emb84","emb85","emb86","emb87","emb88","emb89","emb90","emb91","emb92","emb93","emb94","emb95","emb96","emb97","emb98","emb99","emb100","emb101","emb102","emb103","emb104","emb105","emb106","emb107","emb108","emb109","emb110","emb111","emb112","emb113","emb114","emb115","emb116","emb117","emb118","emb119","emb120","emb121","emb122","emb123","emb124","emb125","emb126","emb127","emb128","emb129","emb130","emb131","emb132","emb133","emb134","emb135","emb136","emb137","emb138","emb139","emb140","emb141","emb142","emb143","emb144","emb145","emb146","emb147","emb148","emb149","emb150","emb151","emb152","emb153","emb154","emb155","emb156","emb157","emb158","emb159","emb160","emb161","emb162","emb163","emb164","emb165","emb166","emb167","emb168","emb169","emb170","emb171","emb172","emb173","emb174","emb175","emb176","emb177","emb178","emb179","emb180","emb181","emb182","emb183","emb184","emb185","emb186","emb187","emb188","emb189","emb190","emb191","emb192","emb193","emb194","emb195","emb196","emb197","emb198","emb199","emb200","emb201","emb202","emb203","emb204","emb205","emb206","emb207","emb208","emb209","emb210","emb211","emb212","emb213","emb214","emb215","emb216","emb217","emb218","emb219","emb220","emb221","emb222","emb223","emb224","emb225","emb226","emb227","emb228","emb229","emb230","emb231","emb232","emb233","emb234","emb235","emb236","emb237","emb238","emb239","emb240","emb241","emb242","emb243","emb244","emb245","emb246","emb247","emb248","emb249","emb250","emb251","emb252","emb253","emb254","emb255","emb256","emb257","emb258","emb259","emb260","emb261","emb262","emb263","emb264","emb265","emb266","emb267","emb268","emb269","emb270","emb271","emb272","emb273","emb274","emb275","emb276","emb277","emb278","emb279","emb280","emb281","emb282","emb283","emb284","emb285","emb286","emb287","emb288","emb289","emb290","emb291","emb292","emb293","emb294","emb295","emb296","emb297","emb298","emb299","emb300","emb301","emb302","emb303","emb304","emb305","emb306","emb307","emb308","emb309","emb310","emb311","emb312","emb313","emb314","emb315","emb316","emb317","emb318","emb319","emb320","emb321","emb322","emb323","emb324","emb325","emb326","emb327","emb328","emb329","emb330","emb331","emb332","emb333","emb334","emb335","emb336","emb337","emb338","emb339","emb340","emb341","emb342","emb343","emb344","emb345","emb346","emb347","emb348","emb349","emb350","emb351","emb352","emb353","emb354","emb355","emb356","emb357","emb358","emb359","emb360","emb361","emb362","emb363","emb364","emb365","emb366","emb367","emb368","emb369","emb370","emb371","emb372","emb373","emb374","emb375","emb376","emb377","emb378","emb379","emb380","emb381","emb382","emb383","emb384","emb385","emb386","emb387","emb388","emb389","emb390","emb391","emb392","emb393","emb394","emb395","emb396","emb397","emb398","emb399","emb400","emb401","emb402","emb403","emb404","emb405","emb406","emb407","emb408","emb409","emb410","emb411","emb412","emb413","emb414","emb415","emb416","emb417","emb418","emb419","emb420","emb421","emb422","emb423","emb424","emb425","emb426","emb427","emb428","emb429","emb430","emb431","emb432","emb433","emb434","emb435","emb436","emb437","emb438","emb439","emb440","emb441","emb442","emb443","emb444","emb445","emb446","emb447","emb448","emb449","emb450","emb451","emb452","emb453","emb454","emb455","emb456","emb457","emb458","emb459","emb460","emb461","emb462","emb463","emb464","emb465","emb466","emb467","emb468","emb469","emb470","emb471","emb472","emb473","emb474","emb475","emb476","emb477","emb478","emb479","emb480","emb481","emb482","emb483","emb484","emb485","emb486","emb487","emb488","emb489","emb490","emb491","emb492","emb493","emb494","emb495","emb496","emb497","emb498","emb499","emb500","emb501","emb502","emb503","emb504","emb505","emb506","emb507","emb508","emb509","emb510","emb511","emb512","emb513","emb514","emb515","emb516","emb517","emb518","emb519","emb520","emb521","emb522","emb523","emb524","emb525","emb526","emb527","emb528","emb529","emb530","emb531","emb532","emb533","emb534","emb535","emb536","emb537","emb538","emb539","emb540","emb541","emb542","emb543","emb544","emb545","emb546","emb547","emb548","emb549","emb550","emb551","emb552","emb553","emb554","emb555","emb556","emb557","emb558","emb559","emb560","emb561","emb562","emb563","emb564","emb565","emb566","emb567","emb568","emb569","emb570","emb571","emb572","emb573","emb574","emb575","emb576","emb577","emb578","emb579","emb580","emb581","emb582","emb583","emb584","emb585","emb586","emb587","emb588","emb589","emb590","emb591","emb592","emb593","emb594","emb595","emb596","emb597","emb598","emb599","emb600","emb601","emb602","emb603","emb604","emb605","emb606","emb607","emb608","emb609","emb610","emb611","emb612","emb613","emb614","emb615","emb616","emb617","emb618","emb619","emb620","emb621","emb622","emb623","emb624","emb625","emb626","emb627","emb628","emb629","emb630","emb631","emb632","emb633","emb634","emb635","emb636","emb637","emb638","emb639","emb640","emb641","emb642","emb643","emb644","emb645","emb646","emb647","emb648","emb649","emb650","emb651","emb652","emb653","emb654","emb655","emb656","emb657","emb658","emb659","emb660","emb661","emb662","emb663","emb664","emb665","emb666","emb667","emb668","emb669","emb670","emb671","emb672","emb673","emb674","emb675","emb676","emb677","emb678","emb679","emb680","emb681","emb682","emb683","emb684","emb685","emb686","emb687","emb688","emb689","emb690","emb691","emb692","emb693","emb694","emb695","emb696","emb697","emb698","emb699","emb700","emb701","emb702","emb703","emb704","emb705","emb706","emb707","emb708","emb709","emb710","emb711","emb712","emb713","emb714","emb715","emb716","emb717","emb718","emb719","emb720","emb721","emb722","emb723","emb724","emb725","emb726","emb727","emb728","emb729","emb730","emb731","emb732","emb733","emb734","emb735","emb736","emb737","emb738","emb739","emb740","emb741","emb742","emb743","emb744","emb745","emb746","emb747","emb748","emb749","emb750","emb751","emb752","emb753","emb754","emb755","emb756","emb757","emb758","emb759","emb760","emb761","emb762","emb763","emb764","emb765","emb766","emb767","emb768","emb769","emb770","emb771","emb772","emb773","emb774","emb775","emb776","emb777","emb778","emb779","emb780","emb781","emb782","emb783","emb784","emb785","emb786","emb787","emb788","emb789","emb790","emb791","emb792","emb793","emb794","emb795","emb796","emb797","emb798","emb799","emb800","emb801","emb802","emb803","emb804","emb805","emb806","emb807","emb808","emb809","emb810","emb811","emb812","emb813","emb814","emb815","emb816","emb817","emb818","emb819","emb820","emb821","emb822","emb823","emb824","emb825","emb826","emb827","emb828","emb829","emb830","emb831","emb832","emb833","emb834","emb835","emb836","emb837","emb838","emb839","emb840","emb841","emb842","emb843","emb844","emb845","emb846","emb847","emb848","emb849","emb850","emb851","emb852","emb853","emb854","emb855","emb856","emb857","emb858","emb859","emb860","emb861","emb862","emb863","emb864","emb865","emb866","emb867","emb868","emb869","emb870","emb871","emb872","emb873","emb874","emb875","emb876","emb877","emb878","emb879","emb880","emb881","emb882","emb883","emb884","emb885","emb886","emb887","emb888","emb889","emb890","emb891","emb892","emb893","emb894","emb895","emb896","emb897","emb898","emb899","emb900","emb901","emb902","emb903","emb904","emb905","emb906","emb907","emb908","emb909","emb910","emb911","emb912","emb913","emb914","emb915","emb916","emb917","emb918","emb919","emb920","emb921","emb922","emb923","emb924","emb925","emb926","emb927","emb928","emb929","emb930","emb931","emb932","emb933","emb934","emb935","emb936","emb937","emb938","emb939","emb940","emb941","emb942","emb943","emb944","emb945","emb946","emb947","emb948","emb949","emb950","emb951","emb952","emb953","emb954","emb955","emb956","emb957","emb958","emb959","emb960","emb961","emb962","emb963","emb964","emb965","emb966","emb967","emb968","emb969","emb970","emb971","emb972","emb973","emb974","emb975","emb976","emb977","emb978","emb979","emb980","emb981","emb982","emb983","emb984","emb985","emb986","emb987","emb988","emb989","emb990","emb991","emb992","emb993","emb994","emb995","emb996","emb997","emb998","emb999","emb1000","emb1001","emb1002","emb1003","emb1004","emb1005","emb1006","emb1007","emb1008","emb1009","emb1010","emb1011","emb1012","emb1013","emb1014","emb1015","emb1016","emb1017","emb1018","emb1019","emb1020","emb1021","emb1022","emb1023","emb1024","Yval"])
    cw.writerows(cr)
# header = "\"class\","
# for i in range(1,1025):
#     header+="\"emb"+str(i)+"\""+","
# header+="\"Yval\""
# print(header)

In [28]:
# fit autogluon models
#all_data = TabularDataset(f'../output/instance_prediction_datasets/250000/{property_label}.{orientation}.transE.embeddings.{limit}.commonclasses.tsv')
all_data = TabularDataset(f'../output/instance_prediction_datasets/250000/{property_label}.{orientation}.text.embeddings.{limit}.tsv')
print(int(len(all_data)*.75))
train_data = all_data[:int(len(all_data)*.75)]
test_data = all_data[int(len(all_data)*.75):]
save_path = f'../embeddings/models/agModels.{property_label}.{orientation}.commonclasses.250000'
label = 'Yval'
print("Summary of class variable: \n", train_data[label].describe())
#predictor = TabularPredictor(label=label, path=save_path).fit(train_data, hyperparameters={'NN':{},'GBM':{},'XT':{},'KNN':{}})

Loaded data from: ../output/instance_prediction_datasets/250000/P19.objects.text.embeddings.full.tsv | Columns = 1026 / 1026 | Rows = 219940 -> 219940


164955
Summary of class variable: 
 count      164955
unique       2756
top       Q486972
freq        12854
Name: Yval, dtype: object


In [29]:
# build test data and evaluate models
save_path = f'../embeddings/models/agModels.{property_label}.{orientation}.text.embeddings.250000'
predictor = TabularPredictor.load(save_path)
y_test = test_data[label]
test_data_nolab = test_data.drop(columns=[label])
print(predictor.get_model_best())
y_pred = predictor.predict(test_data_nolab)
# perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)
# predictor.leaderboard(test_data, silent=True)

WeightedEnsemble_L2


In [52]:
from collections import Counter

def most_frequent(List):
    return max(set(List), key = List.count)

incorrect = {}
correct = {}
counts = {}
y_test_list = list(y_test)
y_pred_list = list(y_pred)
for index, value in enumerate(y_test_list):
    if value != y_pred_list[index]:
        if value in incorrect:
            incorrect[value].append(y_pred_list[index])
        else:
            incorrect[value] = [y_pred_list[index]]
    else:
        if value in correct:
            correct[value].append(y_pred_list[index])
        else:
            correct[value] = [y_pred_list[index]]
for key in incorrect:
    correctcount = 0
    if key in correct:
        correctcount = len(correct[key])
    counts[key] = len(incorrect[key])/((len(incorrect[key])+correctcount))
filtered_counted = {}
for key in counts:
    correctcount = 0
    if key in correct:
        correctcount = len(correct[key])
    if (len(incorrect[key])+correctcount) > 50:
        filtered_counted[key] = counts[key]
print({k:v for k, v in sorted(filtered_counted.items(), key=lambda item: item[1], reverse=True)})
for key in incorrect:
    print(key, most_frequent(incorrect[key]))
#print(correct)
# occurence_count = Counter(y_pred_list)
# print(occurence_count)
occurence_count = Counter(y_test_list)
print(occurence_count)

{'Q13415368': 1.0, 'Q13415366': 1.0, 'Q17143371': 1.0, 'Q15303838': 0.9402985074626866, 'Q13414763': 0.8823529411764706, 'Q1637706': 0.8571428571428571, 'Q902814': 0.7894736842105263, 'Q82794': 0.7, 'Q15284': 0.6194690265486725, 'Q149621': 0.5833333333333334, 'Q21869758': 0.5822784810126582, 'Q667509': 0.5333333333333333, 'Q1549591': 0.5205047318611987, 'Q79007': 0.5, 'Q20871353': 0.4946236559139785, 'Q618123': 0.4915254237288136, 'Q7216840': 0.48598130841121495, 'Q2023000': 0.47297297297297297, 'Q4224624': 0.4659090909090909, 'Q192287': 0.4558258642765685, 'Q1115575': 0.447887323943662, 'Q1852859': 0.43548387096774194, 'Q15334': 0.41935483870967744, 'Q56061': 0.417910447761194, 'Q3024240': 0.4098360655737705, 'Q515': 0.3828647925033467, 'Q13217644': 0.38181818181818183, 'Q2630741': 0.3670886075949367, 'Q15630849': 0.34615384615384615, 'Q34841063': 0.32727272727272727, 'Q2983893': 0.31896551724137934, 'Q856076': 0.3148148148148148, 'Q13410438': 0.3088235294117647, 'Q3957': 0.3025291828

Q475061 Q1077097
Q285181 Q532
Q1208802 Q748149
Q97312698 Q15727673
Q181307 Q3257686
Q1070990 Q1065118
Q355567 Q3024240
Q4312761 Q86622
Q1372205 Q253019
Q603715 Q3957
Q34982659 Q659103
Q19644586 Q123705
Q4414032 Q532
Q19832712 Q1289426
Q18920569 Q618123
Q1860467 Q202595
Q37800986 Q515
Q3505887 Q515
Q15127838 Q12131640
Q16858213 Q640364
Q7930612 Q7830213
Q3348196 Q21672098
Q559618 Q2039348
Q555937 Q23442
Q25412766 Q1840161
Q15634846 Q490329
Q1075520 Q953822
Q13414765 Q1093829
Q19832914 Q1093829
Q131596 Q3957
Q16887380 Q484170
Q765865 Q917092
Q13410447 Q1093829
Q54050 Q532
Q1547289 Q42744322
Q15643578 Q3257686
Q2177636 Q3957
Q1963041 Q515
Q15901936 Q490329
Q18545121 Q490329
Q18545295 Q490329
Q465842 Q1639634
Q47168 Q19953632
Q1343246 Q1115575
Q13410438 Q15127012
Q13415367 Q2154459
Q28659128 Q1549591
Q5098 Q3191695
Q59081206 Q34841063
Q192299 Q18663579
Q83640434 Q18663579
Q659103 Q902814
Q13414758 Q15127012
Q17268368 Q2706302
Q13410524 Q1093829
Q15141625 Q1077097
Q64578911 Q16917
Q15731904